# IML 18 - Project
## Task 1b: Linear Regression

In [253]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

from sklearn.decomposition import PCA
from sklearn.base import clone
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [254]:
# Load training data CSV using Pandas
filename = 'train.csv'
data = pd.read_csv(filename, header=0)
# Separate training data into Y and X
array = data.values
Y = array[:, 1]
X = array[:, 2:7]

In [255]:
# Split data into own train and test sets
test_size = 0.1
seed = 13
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=test_size,
                                                    random_state=seed)

In [256]:
# define feature transformation as given by task description
def feature_transform(X):
    x1, x2, x3, x4, x5 = X[:,0], X[:,1], X[:,2], X[:,3], X[:,4] 
    return np.column_stack((
                     x1,
                     x2,
                     x3,
                     x4,
                     x5,
                     x1**2,
                     x2**2,
                     x3**2,
                     x4**2,
                     x5**2,
                     np.exp(x1),
                     np.exp(x2),
                     np.exp(x3),
                     np.exp(x4),
                     np.exp(x5),
                     np.cos(x1),
                     np.cos(x2),
                     np.cos(x3),
                     np.cos(x4),
                     np.cos(x5),
                     np.ones_like(x1)
                     ))

In [257]:
# Make pipeline: mandatory transformations plus other experiments
pipeline = Pipeline([
#     ('std_scaler', StandardScaler()), # can't use as test set not handed out
     ('feature_transformer', FunctionTransformer(feature_transform)),
#     ('PCA', PCA()) # can't use as test set not handed out
    ])

In [258]:
# Transform train and test sets with pipeline
X_train_prepared = pipeline.fit_transform(X_train)
X_test_prepared = pipeline.transform(X_test)

In [259]:
# Experiment: Test across a wide selection of models to get a feel for the data
# Create list of models to be tested
models = []
# LinearRegressor for baseline
models.append(('linear_reg',
              LinearRegression(),
              {}
              ))
# # XGBRegressor for feature selection
# models.append(('xgboost_reg',
#              XGBRegressor(subsample = 0.5,
#              objective = 'reg:linear',
#              num_estimators = 1000,
#              learning_rate = 0.2),
#              [{'max_depth': [5, 10, 20, 30],
#                'min_child_weight': [1, 3, 5, 8, 10],
#                'colsample_bytree': [0.6, 0.8, 1.0]}]
#              ))
# # RandomForestRegressor for feature selection
# models.append(('forest_reg',
#               RandomForestRegressor(random_state=42),
#               [{'bootstrap': [True, False],
#               'n_estimators': [10, 20, 40],
#               'max_features': [2, 4, 8, 16, 21]}]
#               ))
# # ExtraTreesRegressor for feature selection
# models.append(('extraTrees_reg',
#               ExtraTreesRegressor(n_estimators=50, max_features=0.5,
#                                   bootstrap=True, oob_score=True,
#                                   n_jobs=-1, random_state=42),
#               {}
#               ))

# models.append(('lasso_reg',
#               Lasso(max_iter=1e6, tol=1e-5, random_state=42),
#               {}
#               ))
    
models.append(('elastic_net',
              ElasticNet(max_iter=1e6, tol=1e-4,
                         random_state=42, selection='random'),
              [{'alpha': [1.0],
              'l1_ratio': [0.4, 0.8, 1.0]}]
              ))
        
# models.append(('svm_reg',
#               SVR(max_iter=-1, tol=1e-5, verbose=0),
#               [{'C': [0.1, 1, 10],
#               'epsilon': [0.1, 0.3, 0.5],
#               'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}]
#               ))

# models.append(('sgd_reg',
#               SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=5e-4,
#               fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
#               loss='huber', max_iter=1e4, n_iter=None, penalty='elasticnet',
#               power_t=0.25, random_state=None, shuffle=True, tol=1e-3,
#               verbose=0, warm_start=True),
#               {}
#               ))

# models.append(('linear_svm',
#               LinearSVR(dual=True, fit_intercept=True,
#                         intercept_scaling=1.0, max_iter=1e6,
#                         random_state=42, tol=1e-5, verbose=0),
#               [{'C': [10],
#               'loss': ['epsilon_insensitive','squared_epsilon_insensitive'],
#               'epsilon': [0.9, 1.8, 3.6]}]
#               ))

In [260]:
# From above: Collect models' results of grid search with cross validation
results = {}
for (name, model, param_grid) in models:
    grid_search = GridSearchCV(model,
                           param_grid, n_jobs=-1, cv=5,
                           scoring='neg_mean_squared_error',
                           verbose=1, return_train_score=True)
    grid_search.fit(X_train_prepared, Y_train)
    pred = grid_search.best_estimator_.predict(X_test_prepared)
    rmse = np.sqrt(mean_squared_error(Y_test, pred))
    print('{0} RMSE: {1:.3}'.format(name, rmse))
    results[name] = [grid_search.best_estimator_, rmse]
    print(60*'.')

print("Overall ranking")
print("Estimator: \t RMSE:")
print(22*'-')
for (name, value) in sorted(results.items(), key=lambda x:x[1][1]):
    print('{0} \t {1:.3}'.format(name, value[1]))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
linear_reg RMSE: 10.9
............................................................
Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished


elastic_net RMSE: 11.9
............................................................
Overall ranking
Estimator: 	 RMSE:
----------------------
linear_reg 	 10.9
elastic_net 	 11.9


[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    0.1s finished


In [171]:
# Experiment: Automatic feature selection by XGBoost
feature_select = SelectFromModel(results['xgboost_reg'][0], prefit=True)
X_train_new = feature_select.transform(X_train_prepared)
X_test_new = feature_select.transform(X_test_prepared)
print(X_train_new.shape)
print(X_test_new.shape)

(810, 10)
(90, 10)


In [262]:
# Experiment: ElasticNetCV to estimate ElasticNet base performance
elastic_cv = ElasticNetCV(
                          cv=5,
                          eps=0.001,
                          fit_intercept=True,
                          l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                          max_iter=1e6,
                          n_alphas=1000,
                          n_jobs=-1,
                          normalize=False,
                          precompute='auto',
                          random_state=42,
                          selection='cyclic',
                          tol=1e-4,
                          verbose=0)
elastic_cv.fit(X_train_prepared, Y_train)
pred = elastic_cv.predict(X_test_prepared)
rmse = np.sqrt(mean_squared_error(Y_test, pred))
print('RMSE: {0:.3}'.format(rmse))
print(elastic_cv.coef_)

RMSE: 10.9
[-0.66947449  3.09220664 -2.75039943  6.48302226 -3.25570265 -0.59670293
  0.23511655  0.          1.01601232 -0.          0.0932698  -3.53037198
  0.16434826 -0.05712628  0.01665568  0.         -0.          0.
 -0.          0.          0.        ]


In [274]:
# Chosen approach: A manually constructed bag of ElasticNetCVs
w_coeff = np.zeros(X_train_prepared.shape[1], dtype='float64')
n_estimators = 20
sample_ratio = 0.9
n_samples = X_train_prepared.shape[0]
batch_size = int(sample_ratio * n_samples)
for i in range(n_estimators):
    sample_rows = np.random.choice(X_train_prepared.shape[0],
                               batch_size, replace=False)
    batch_X_train_prepared = X_train_prepared[sample_rows, :]
    batch_Y_train = Y_train[sample_rows]
    elastic_cv = ElasticNetCV(
                          cv=5,
                          eps=0.001,
                          fit_intercept=True,
                          l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                          max_iter=1e6,
                          n_alphas=1000,
                          n_jobs=-1,
                          normalize=False,
                          precompute='auto',
                          random_state=42,
                          selection='cyclic',
                          tol=1e-4,
                          verbose=0)
    elastic_cv.fit(batch_X_train_prepared, batch_Y_train)
    pred = elastic_cv.predict(X_test_prepared)
    rmse = np.sqrt(mean_squared_error(Y_test, pred))
    print('Estimator {0} RMSE: {1:.3}'.format(i, rmse))
    w_coeff += elastic_cv.coef_
w_coeff /= n_estimators

error=0
for i in range(X_test_prepared.shape[0]):
    error += (w.dot(X_test_prepared[i])-Y_test[i])**2
rmse=np.sqrt(error/n)
print('Bag of {0} estimators RMSE: {1:.3}'.format(n_estimators, rmse))

Estimator 0 RMSE: 11.0
Estimator 1 RMSE: 10.9
Estimator 2 RMSE: 11.0
Estimator 3 RMSE: 11.0
Estimator 4 RMSE: 11.0
Estimator 5 RMSE: 10.9
Estimator 6 RMSE: 11.1
Estimator 7 RMSE: 10.9
Estimator 8 RMSE: 11.1
Estimator 9 RMSE: 11.1
Estimator 10 RMSE: 11.0
Estimator 11 RMSE: 11.0
Estimator 12 RMSE: 11.1
Estimator 13 RMSE: 11.1
Estimator 14 RMSE: 10.9
Estimator 15 RMSE: 11.0
Estimator 16 RMSE: 10.9
Estimator 17 RMSE: 11.0
Estimator 18 RMSE: 11.1
Estimator 19 RMSE: 10.9
Bag of 20 estimators RMSE: 12.6


In [269]:
# Experiment: BaggingRegressor on ElasticNetCV to get OOB score
elastic_bagging = BaggingRegressor(base_estimator=ElasticNetCV(cv=5,
                                          eps=0.001,
                                          fit_intercept=True,
                                          l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
                                          max_iter=1e6,
                                          n_alphas=1000,
                                          n_jobs=-1,
                                          normalize=False,
                                          precompute='auto',
                                          random_state=42,
                                          selection='cyclic',
                                          tol=1e-4,
                                          verbose=0),
                                  n_estimators=20,
                                  max_samples=0.9,
                                  max_features=0.9,
                                  bootstrap=True,
                                  bootstrap_features=False,
                                  oob_score=True,
                                  warm_start=False,
                                  n_jobs=-1,
                                  random_state=42,
                                  verbose=0)
elastic_bagging.fit(X_train_prepared, Y_train)
pred = elastic_bagging.predict(X_test_prepared)
rmse = np.sqrt(mean_squared_error(Y_test, pred))
oob = elastic_bagging.oob_score_
print('RMSE: {0:.3}, OOB score: {1:.3}'.format(rmse, oob))

RMSE: 11.0, OOB score: 0.827


In [277]:
# Decide on final model and write to file
# final_model = None
result = w_coeff
# final_pred = final_model.predict(X_test_prepared)
# final_rmse = np.sqrt(mean_squared_error(Y_test, final_pred))
# print("Final RMSE: {:.3}".format(final_rmse))
filename = 'result.csv'
final_result = pd.DataFrame(result)
final_result.to_csv(filename, header=False, index=False) 